In [125]:
import glob, json, pathlib
from tqdm.notebook import tqdm
sanitychecks = list(glob.glob(r"D:\ig_pipeline\cad\*\*\artifacts\sanitycheck.json"))
jsons = {}
for x in tqdm(sanitychecks):
    with open(x, "r") as f:
        name = "/".join(pathlib.Path(x).parts[-4:-2])
        jsons[name] = json.load(f)

  0%|          | 0/90 [00:00<?, ?it/s]

In [126]:
total = len(jsons)
failed = {name for name, x in jsons.items() if x["errors"]}
total, len(failed)

(90, 51)

In [129]:
from collections import defaultdict
import re
error_templates = [
    "Category (.*) for object .* does not exist on spreadsheet.",
    "(.*) has too many vertices: .* > .*",
    "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*.",
    "(.*) is missing in instance .*, so relative transform check cannot be completed.",
    "(.*) has bad name.",
    "(.*) has negative object offset scale.*",
    "(.*) has scale that is not 1. Reset scale.",
    ".*No instance ID 0 instance of (.*)",
    "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script.",
    "All instances of (.*) do not have contiguous instance IDs. Missing: .*",
    "(.*) has disallowed type.*",
    "Light object (.*) should not have object offset rotation. Reset pivot.",
    "(.*) has different pivot offset position \(by (.*)\). Match pivots on each instance.",
    "(.*) has different pivot offset rotation \(by (.*)\). Match pivots on each instance.",
    "(.*) has different shear. Match scaling axes on each instance.",
    "Model ID (.*) contains 'todo'.",
    "(.*) should not have an upper side.",
    "(.*) should have an upper side.",
    "(.*) has different vertex count than recorded in provider.*",
    "(.*) has different face count than recorded in provider.*",
    "(.*) has no collision mesh. Create a collision mesh.",
    "(.*) has different UV unwrapping than recorded. Reunwrap the object.",
    "Inconsistent link sets within model (.*)",
    "(.*) element .* has too many vertices .*",
    "Model ID (.*) requires joints but has no joints.",
    "Expected meta links for (.*) are missing: .*",
    "Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.",
    "Cannot validate meta links for (.*): Category haystack not found in taxonomy.",
    "Cannot validate joints for model ID (.*): Category .* not found in taxonomy.",
    "Cannot validate clothness: category (.*) not found in taxonomy.", # TODO: fix this to include model ID
    "(.*) element .* is not a volume",
    "(.*) element .* has elements trimesh still finds splittable",
]
error_matches = defaultdict(set)
error_matches_by_file = defaultdict(lambda: defaultdict(set))
unknown_error_files = []
for name, f in jsons.items():
    unknown_error = False
    for error in f["errors"] + f["warnings"]:
        for i, tmpl in enumerate(error_templates):
            match = re.fullmatch(tmpl, error, re.S)
            if match is not None:
                error_matches[tmpl].add(match.group(1))
                error_matches_by_file[tmpl][name].add(match.group(1))

                break
        else:
            print(error)
            unknown_error = True
    if unknown_error:
        unknown_error_files.append(name)
        
print("\nCounts")
for tmpl, objs in error_matches.items():
    print(f"{tmpl}: {len(objs)}")

print("\n\nCounts by file")
for tmpl, matches_by_file in error_matches_by_file.items():
    all_matches = error_matches[tmpl]
    if not all_matches:
        continue
    print(f"\n{tmpl}")
    for name, matches in sorted(matches_by_file.items(), key=lambda x: x[1], reverse=True):
        if not matches:
            break
        print(f"  {name}: {len(matches)}")

Cannot validate meta links for L-haystack-lujosc-0: Category haystack not found in taxonomy.
Cannot validate meta links for salad_bowl-dhdhul-0-base_link: Category salad_bowl not found in taxonomy.
Cannot validate meta links for cup-wbqjbt-0-base_link: Category cup not found in taxonomy.
Cannot validate meta links for diced__beef_roast-vrzhwc-0: Category diced__beef_roast not found in taxonomy.

Counts
Model ID (.*) requires joints but has no joints.: 9
Model ID (.*) contains 'todo'.: 7
Expected meta links for (.*) are missing: .*: 995
Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 60
(.*) has too many vertices: .* > .*: 123
(.*) element .* has too many vertices .*: 1
Cannot validate joints for model ID (.*): Category .* not found in taxonomy.: 4
Cannot validate clothness: category (.*) not found in taxonomy.: 4
(.*) has no collision mesh. Create a collision mesh.: 14
Articulated object (.*) instances have different scales for base links. This may

In [119]:
# Print the number of kinds of error for each target
IGNORE_ERRORS = {
    "(.*) has too many vertices: .* > .*",
    "(.*) element .* has too many vertices .*",
    "(.*) has no collision mesh. Create a collision mesh.",
}
print("\n\nCounts by error")
for name in jsons.keys():
  all_errors = {}
  for tmpl, matches_by_file in error_matches_by_file.items():
      if tmpl in IGNORE_ERRORS:
          continue
      matches = matches_by_file[name]
      if not matches:
          continue
      all_errors[tmpl] = len(matches)

  if all_errors:
      print(f"\n{name}")
      for k, v in sorted(all_errors.items(), key=lambda x: x[1], reverse=True):
          print(f"  {k}: {v}")



Counts by error

objects/batch-00
  Expected meta links for (.*) are missing: .*: 16
  Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 2
  Model ID (.*) contains 'todo'.: 1
  Model ID (.*) requires joints but has no joints.: 1

objects/batch-01
  Expected meta links for (.*) are missing: .*: 15
  Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 2

objects/batch-02
  Expected meta links for (.*) are missing: .*: 13
  Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 11
  Model ID (.*) requires joints but has no joints.: 1

objects/batch-03
  Expected meta links for (.*) are missing: .*: 14
  Cloth object (.*) should consist of exactly 1 element. Currently it has .* elements.: 3
  Model ID (.*) requires joints but has no joints.: 2

objects/batch-04
  Expected meta links for (.*) are missing: .*: 9
  Model ID (.*) requires joints but has no joints.: 2
  Cloth object (.*) shoul

In [120]:
vertex_issues = error_matches_by_file["(.*) has different vertex count than recorded in provider.*"]

for filename, ids in sorted(vertex_issues.items(), key=lambda x: len(x[1]), reverse=True):
  print(f"{filename}: {len(ids)}. ({', '.join(ids)})")

scenes/office_large: 2. (immwzb-1, immwzb-0)
scenes/school_geography: 1. (khxpgc-1)
objects/batch-00: 0. ()
objects/batch-01: 0. ()
objects/batch-02: 0. ()
objects/batch-03: 0. ()
objects/batch-04: 0. ()
objects/batch-05: 0. ()
objects/batch-06: 0. ()
objects/batch-07: 0. ()
objects/batch-08: 0. ()
objects/batch-09: 0. ()
objects/batch-10: 0. ()
objects/batch-11: 0. ()
objects/batch-12: 0. ()
objects/batch-13: 0. ()
objects/custom-aa: 0. ()
objects/legacy_batch-00: 0. ()
objects/legacy_batch-01: 0. ()
objects/legacy_batch-02: 0. ()
objects/legacy_batch-03: 0. ()
objects/legacy_batch-04: 0. ()
objects/legacy_batch-05: 0. ()
objects/legacy_batch-06: 0. ()
objects/legacy_batch-07: 0. ()
objects/legacy_batch-08: 0. ()
objects/legacy_batch-09: 0. ()
objects/legacy_batch-10: 0. ()
objects/legacy_batch-11: 0. ()
objects/legacy_batch-12: 0. ()
objects/procedural-aa: 0. ()
objects/substances-01: 0. ()
objects/substances-02: 0. ()
objects/task_relavant-ab: 0. ()
scenes/Beechwood_0_garden: 0. ()


In [121]:
from collections import defaultdict
import re

warning_re = re.compile(r"(.*) has too many vertices: (\d+) > 20000")
warn_objs = defaultdict(dict)
for name, x in jsons.items():
    for w in x["warnings"]:
        m = warning_re.fullmatch(w)
        if m:
            warn_objs[name][m.group(1)] = int(m.group(2))
            
print(warn_objs)

defaultdict(<class 'dict'>, {'objects/batch-00': {'L-tag-ohegnp-0': 21794, 'chicken_wire-czoiqw-0': 52305}, 'objects/batch-01': {'L-shoe_rack-spdrzb-0': 59456}, 'objects/batch-02': {'L-radio-vgioak-0': 50123}, 'objects/batch-06': {'zipper-qwsvid-0': 27426}, 'objects/legacy_batch-01': {'swivel_chair-fqbnuk-0-base_link': 36405, 'swivel_chair-lizuow-0-base_link': 25743, 'swivel_chair-lxupys-0-base_link': 35465, 'swivel_chair-mqxrim-0-base_link': 21060, 'swivel_chair-opkoam-0-base_link': 38343}, 'objects/legacy_batch-02': {'table_lamp-pmekkr-0-arm-body-R-lower': 23529, 'table_lamp-pmekkr-0-head-neck-R-lower': 44331, 'table_lamp-pmekkr-0-arm-body-R-upper': 23529, 'table_lamp-pmekkr-0-head-neck-R-upper': 44331, 'table_lamp-pxdhzm-0-head-arm-R-lower': 33474, 'table_lamp-pxdhzm-0-head-arm-R-upper': 33474}, 'objects/legacy_batch-03': {'desk-egzdgz-0-base_link': 20943, 'dishwasher-znfvmj-0-link_0-base_link-P-lower': 31812, 'dishwasher-znfvmj-0-link_1-base_link-P-lower': 31812, 'dishwasher-znfvmj

In [122]:
# Worst offenders
sorted((cnt, obj) for warn_target in warn_objs.values() for obj, cnt in warn_target.items())

[(20082, 'barbecue_sauce_bottle-gfxrnj-0'),
 (20082, 'catsup_bottle-qfvqfm-0'),
 (20082, 'chocolate_sauce_bottle-yegrkf-0'),
 (20082, 'glaze_bottle-zdxagk-0'),
 (20082, 'hot_sauce_bottle-qvpthd-0'),
 (20082, 'mustard_bottle-lgxfyv-0'),
 (20082, 'salsa_bottle-kydilb-0'),
 (20082, 'soy_sauce_bottle-saujjl-0'),
 (20082, 'squeeze_bottle-uzmhdn-0'),
 (20082, 'sugar_syrup_bottle-kdlbbq-0'),
 (20082, 'vanilla_bottle-drevku-0'),
 (20082, 'white_sauce_bottle-gtwngf-0'),
 (20082, 'wine_sauce_bottle-vqtevv-0'),
 (20279, 'pot_plant-kxmvco-0-base_link'),
 (20943, 'desk-egzdgz-0-base_link'),
 (21060, 'swivel_chair-mqxrim-0-base_link'),
 (21794, 'L-tag-ohegnp-0'),
 (22097, 'L-garden_chair-deuron-0'),
 (22097, 'L-garden_chair-deuron-1'),
 (22560, 'room_light-fxtqtn-0'),
 (22560, 'room_light-fxtqtn-1'),
 (22740, 'fridge-jtqazu-0-base_link'),
 (22956, 'L-pedestal_table-ifmhpn-0'),
 (22956, 'L-pedestal_table-ifmhpn-1'),
 (22998, 'C-stockpot-gxiqbw-0'),
 (22998, 'C-stockpot-gxiqbw-1'),
 (23465, 'floor_lam

In [123]:
# Check which of the "articulated object has diff scale" errors are on bad objects
providers = json.loads(pathlib.Path(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json").read_text())["providers"]
id_providers = {k.split("-")[-1]: v for k, v in providers.items()}
categories = dict([tuple(reversed(k.split("-"))) for k in providers.keys()])
articulation_tmpl = "Articulated object (.*) instances have different scales for base links. This may have broken things during the match links script."
for target, objs in error_matches_by_file[articulation_tmpl].items():
    if not objs:
        continue
    bad_objs = set()
    for obj in objs:
        if id_providers[obj] != target:
            name = f"{categories[obj]}-{obj}"
            bad_objs.add(name)

    if not bad_objs:
        continue
    
    print(f"{target}: {len(bad_objs)} {bad_objs}")

scenes/Beechwood_0_garden: 6 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet-immwzb', 'door-lvgliq', 'bottom_cabinet_no_top-spojpj'}
scenes/Beechwood_0_int: 7 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet_no_top-qohxjq', 'bottom_cabinet-immwzb', 'door-lvgliq', 'door-ohagsq', 'bottom_cabinet_no_top-spojpj'}
scenes/Beechwood_1_int: 9 {'top_cabinet-dmwxyl', 'window-ulnafj', 'bottom_cabinet-kubcdk', 'bottom_cabinet-jhymlr', 'door-lvgliq', 'door-ohagsq', 'window-ithrgo', 'bottom_cabinet_no_top-pluwfl', 'toilet-kfmkbm'}
scenes/Benevolence_0_int: 1 {'door-lvgliq'}
scenes/Benevolence_1_int: 6 {'top_cabinet-dmwxyl', 'top_cabinet-eobsmt', 'window-ulnafj', 'cedar_chest-fwstpx', 'bottom_cabinet_no_top-ufhpbn', 'top_cabinet-lsyzkh'}
scenes/Benevolence_2_int: 2 {'window-ithrgo', 'door-lvgliq'}
scenes/hotel_gym_spa: 2 {'door-nfrbch', 'door-ceddpg'}
scenes/house_single_floor: 1 {'bottom_cabinet_no_top-gjrero'}
scenes/Ihlen_0_int: 3 {'door-ktydvs', 'win

In [124]:
# Get a list of all the objects that show up in the match links relevant errors
mismatched_link_offset_tmpl = "(.*) link .* has different .* in instance .* compared to instance .*. .* difference: .*."
objects_for_match_links = error_matches[mismatched_link_offset_tmpl] | error_matches[articulation_tmpl]
print(", ".join(f'"{x}"' for x in sorted(objects_for_match_links)))

"bamfsz", "bmsclc", "ceddpg", "dladgw", "dmwxyl", "dnyzym", "dqnbsj", "ehwmol", "emeeke", "eobsmt", "fexqbj", "fqhdne", "fwstpx", "gemgfz", "ggcyib", "gjrero", "gpkbiw", "ijnwlp", "immwzb", "inmymj", "ithrgo", "jhymlr", "jrhgeu", "kfmkbm", "ktydvs", "kubcdk", "kwbnhy", "lcrefl", "lsyzkh", "lvgliq", "lwjdmj", "nfrbch", "nnvyol", "nwlbit", "ohagsq", "pluwfl", "pwoerr", "pxhbim", "qohxjq", "rgmujm", "rvunhj", "rxvopf", "slgzfc", "spojpj", "ufhpbn", "ulnafj", "uztisk", "wwhydr", "xbfgjc", "xiajyb", "xxipyh", "xyejdx"
